In [73]:
###########################################################################################################################
# Generating listings via LLM
# The code in this cell was used to generate listings. The listings were coppied and pasted in the "Listings.txt" file
# For the rest of this notebook we will read these listings from file
###########################################################################################################################

# from my_opanai_api_helper import generate_listings
# response = generate_listings(5);
# print(response)

In [74]:
######################################################################
# Reading the lsitings from the file "Listings.txt"
######################################################################

import json
def read_listings_from_file(input_file):
  with open(input_file, "r") as read_content: 
    return json.load(read_content)

listings_map = read_listings_from_file("Listings.txt")

In [75]:
######################################################################
# Storing the listings in a vector database, ChromaDb
# Embeddings are automatically generated by ChromaDb
######################################################################

import chromadb

client = chromadb.Client()
#client.delete_collection("real-estate-listings")
collection = client.get_or_create_collection("real-estate-listings")

for listing in listings_map['listings']:

    #document content is the concatenation for House and Neighborhood descriptions
    document_content = "House preferences: " + listing['description'] + " Neighborhood preferences: " + listing['neighborhood description'];

    #metadata map
    doc_meta_data =  {field: listing[field] for field in ['neighborhood', 'price', 'bedrooms', 'bathrooms', 'size_sqft']}

    #add record
    collection.add(
        documents=[document_content], 
        metadatas=[doc_meta_data], 
        ids=[str(listing['id'])]
        )


Insert of existing embedding ID: 1
Add of existing embedding ID: 1


Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 5
Insert of existing embedding ID: 6
Add of existing embedding ID: 6
Insert of existing embedding ID: 7
Add of existing embedding ID: 7
Insert of existing embedding ID: 8
Add of existing embedding ID: 8
Insert of existing embedding ID: 9
Add of existing embedding ID: 9
Insert of existing embedding ID: 10
Add of existing embedding ID: 10
Insert of existing embedding ID: 11
Add of existing embedding ID: 11
Insert of existing embedding ID: 12
Add of existing embedding ID: 12
Insert of existing embedding ID: 13
Add of existing embedding ID: 13
Insert of existing embedding ID: 14
Add of existing embedding ID: 14
Insert of existing embedding ID: 15
Add of existing embedding ID: 15


In [76]:
######################################################################
# Search & Print Results methods
######################################################################
from my_opanai_api_helper import augment_listing
from IPython.display import display, HTML


#This method performs a search in the VectorDb
#It concatenates all user's answers and uses them as a search document   
def do_search(answers):    
    
    document_query = " ".join(answers)
    results = collection.query(
        query_texts=[document_query], 
        n_results=5
    )
    
    return results

#Print the questions and answers
#This shows the inetrcaction between the agent and the potential buyer
def print_q_and_a(questions, answers):
    q_and_a_str = ""
    for i in range(len(questions)):
        q_and_a_str += "Q: " + questions[i] + "\n"
        q_and_a_str += "A: " + answers[i] + "\n\n"
    print(q_and_a_str)


#Print the search results
def print_results(results, answers, include_original_desc):    
    meta_data = results['metadatas'][0]
    documents = results['documents'][0]
    
    cards_html = ""
    for i in range(len(meta_data)):
        cards_html += generate_card_html_str(meta_data[i], documents[i], answers, include_original_desc)

    html_result_component = HTML(f'<div style="display: flex; flex-wrap: wrap;">{cards_html}</div>')
    display(html_result_component)
   
#Print a single search result
def generate_card_html_str(result, document, answers, include_original_desc):

    original_desc = "";
    if(include_original_desc):
        original_desc = f"<p><b>Original Listing Description:</b></br>{document}</p>"

    card_html = f"""
        <div style="border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin: 10px; width: 200px;">
            <h3>{result['neighborhood']}</h3>
            
            <p>${"{:,}".format(result['price'])}</p>
            <p>{result['bedrooms']} bedroom(s)</p>
            <p>{result['bathrooms']} bath</p>
            <p><b>LLM Augmented Description:</b></br>{augment_listing(num_words=50, preferences_list=answers, original_description=document)}</p>
            {original_desc}
        </div>
    """
    return card_html

In [77]:
######################################################################
# Questions & Answers
# We have 1 set of questions and 2 sets of answers for these questions
# The 2 sets of answers are for 2 users with different preferences Urban vs Suburban
# When we run the notebook we can pick one of the users
######################################################################
questions = [   
    "How big do you want your house to be?", 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
            ]

subuarban_answers = [
    "A comfortable four-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and parks for kids.",
    "A backyard for gardening, a two-car garage.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "Suburban for sure. I want a quiet house far from the hustle and bustle."
    ]

urban_answers = [
    "A small house for 1 person. 1 bedroom is good. I would consider a 2 bedroom if the price is right.",
    "Anything in the heart of the city would be great, close to public transport, close to restaurants and nightlife.",
    "A parking spot for one car, and a modern, energy-efficient heating system.",
    "Public transport and bike-friendly roads.",
    "Urban for sure. As urban as it gets."
    ]

In [78]:
######################################################################
# Prompt the user for the user preferences Urban(1) or Suburban(2)
# Display the questions and answers relevant to the user preferences
# Perform the search based on the user's answers
# Print the search results
# Note the search results will include both the original unit description and the LLM Augmented description
######################################################################

#Constants
SUB_URBAN_PREF  = "SUB_URBAN"
URBAN_PREF     = "URBAN"

#Pick user preferences and populate the answers accordingly
answers = []
def pick_preference():
    user_input = input("You can run this notebook as one of 2 potential shoppers with urban or suburban preferences.\n Enter 1 for Urban, Enter 2 for Suburban")    
    global answers
    if(user_input == "1"):
        answers = urban_answers
        print("You have picked a user with urban preferences\n")
    elif(user_input == "2"):
        answers = subuarban_answers
        print("You have picked a user with suburban preferences\n")
    else:
        print("Unknwon preference. Will Exit")
        return False
    return True

#If valid preferences have been picked
if(pick_preference()):
    #Display the Q&A
    print_q_and_a(questions, answers)

    #Perform the search in Vector DB
    results = do_search(answers)

    #Print the results. This will also call the method that augments the results 
    print_results(results, answers, True)

print("Exiting")

You have picked a user with suburban preferences

Q: How big do you want your house to be?
A: A comfortable four-bedroom house with a spacious kitchen and a cozy living room.

Q: What are 3 most important things for you in choosing this property?
A: A quiet neighborhood, good local schools, and parks for kids.

Q: Which amenities would you like?
A: A backyard for gardening, a two-car garage.

Q: Which transportation options are important to you?
A: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.

Q: How urban do you want your neighborhood to be?
A: Suburban for sure. I want a quiet house far from the hustle and bustle.




Exiting
